<a href="https://colab.research.google.com/github/ColdTbrew/pixel_Satellite-Image-Segmentation/blob/main/infer_NB_m2f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir test
%cd test

/content/test


In [3]:
cp -rvf /content/drive/MyDrive/copy .

'/content/drive/MyDrive/copy' -> './copy'
'/content/drive/MyDrive/copy/test_image.tar' -> './copy/test_image.tar'
'/content/drive/MyDrive/copy/sample_submission.csv' -> './copy/sample_submission.csv'
'/content/drive/MyDrive/copy/test.csv' -> './copy/test.csv'
'/content/drive/MyDrive/copy/infer.py' -> './copy/infer.py'
'/content/drive/MyDrive/copy/mask2former_swin-l.py' -> './copy/mask2former_swin-l.py'
'/content/drive/MyDrive/copy/best_mIoU_iter_20000.pth' -> './copy/best_mIoU_iter_20000.pth'


In [5]:
!tar xf "./copy/test_image.tar"

In [6]:
!pip3 install openmim
!mim install mmengine
!mim install "mmcv>=2.0.0"

In [7]:
!git clone https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!git checkout main
!pip install -e .

In [8]:
#mask2former만 적용
!pip install "mmdet>=3.0.0rc4"

In [11]:
%cd /content/test/mmsegmentation

/content/test/mmsegmentation


In [ ]:
import os
import mmcv
import torch
from tqdm.notebook import tqdm
from PIL import Image
import pandas as pd
import numpy as np

from mmseg.apis import init_model, inference_model

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def main():
    save_dir = '/content/drive/MyDrive/csvs'  # save dir
    file_name = 'mask2former_swin_l_01'  # 저장할 파일 이름
    config_path = '/content/test/copy/mask2former_swin-l.py'  # config 파일 위치
    ckpt_path = '/content/test/copy/best_mIoU_iter_20000.pth' # 체크포인트 파일 위치
    sample_path = "/content/test/copy/test.csv"
    submit_path = "/content/test/copy/sample_submission.csv"
    test_image_path = "/content/test/test_image"
    device = "cuda"

    model = init_model(config_path, ckpt_path, device)

    data = pd.read_csv(sample_path)['id'].values.tolist()

    with torch.no_grad():
        model.eval()
        result = []
        for img_id in tqdm(data):
            img_path = os.path.join(test_image_path, img_id + ".png")
            mask = inference_model(model, img_path)
            mask = mask.pred_sem_seg.data
            mask = torch.squeeze(mask).cpu().numpy()
            mask = mask.astype(np.uint8)
            mask = Image.fromarray(mask)
            mask = mask.resize((960, 540), Image.NEAREST)
            mask = np.array(mask)
            for class_id in range(12):
                class_mask = (mask == class_id).astype(np.uint8)
                if np.sum(class_mask) > 0:  # 마스크가 존재하는 경우 encode
                    mask_rle = rle_encode(class_mask)
                    result.append(mask_rle)
                else:  # 마스크가 존재하지 않는 경우 -1
                    result.append(-1)

    submit = pd.read_csv(submit_path)
    submit['mask_rle'] = result
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    submit.to_csv(os.path.join(save_dir, file_name + '.csv'), index=False)
main()


Loads checkpoint by local backend from path: /content/test/copy/best_mIoU_iter_20000.pth


  0%|          | 0/1898 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
